In [1]:
import pandas as pd
import numpy as np
import boto3

In [2]:
import sys
sys.path.append("/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series")
from utils import load_config
config = load_config("../config/development/pipeline.yaml")

In [3]:
# Read data
data = pd.read_csv("../data/raw/train.csv")
reordered_columns = pd.Index(["target"]).append(data.columns.drop("target"))
data = data[reordered_columns]
print("data shape :", data.shape)
data.head(5)

data shape : (2018352, 9)


,target,county,is_business,product_type,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0.713,0,0,1,0,2021-09-01 00:00:00,0,0,0
1,96.590,0,0,1,1,2021-09-01 00:00:00,0,1,0
2,0.000,0,0,2,0,2021-09-01 00:00:00,0,2,1
3,17.314,0,0,2,1,2021-09-01 00:00:00,0,3,1
4,2.904,0,0,3,0,2021-09-01 00:00:00,0,4,2


In [4]:
len(data["data_block_id"].unique())

638

# Missing datetimes

In [5]:
# at each time step, a new value is generated per 'prediction_unit_id'
data = data.sort_values(by=["prediction_unit_id", "datetime"])
data.head()

,target,county,is_business,product_type,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0.713,0,0,1,0,2021-09-01 00:00:00,0,0,0
1,96.590,0,0,1,1,2021-09-01 00:00:00,0,1,0
122,1.132,0,0,1,0,2021-09-01 01:00:00,0,122,0
123,77.691,0,0,1,1,2021-09-01 01:00:00,0,123,0
244,0.490,0,0,1,0,2021-09-01 02:00:00,0,244,0


In [6]:
data.insert(loc=6, column="datetime-1", value=data["datetime"].shift(1))
data.head()

,target,county,is_business,product_type,is_consumption,datetime,datetime-1,data_block_id,row_id,prediction_unit_id
0,0.713,0,0,1,0,2021-09-01 00:00:00,None,0,0,0
1,96.590,0,0,1,1,2021-09-01 00:00:00,2021-09-01 00:00:00,0,1,0
122,1.132,0,0,1,0,2021-09-01 01:00:00,2021-09-01 00:00:00,0,122,0
123,77.691,0,0,1,1,2021-09-01 01:00:00,2021-09-01 01:00:00,0,123,0
244,0.490,0,0,1,0,2021-09-01 02:00:00,2021-09-01 01:00:00,0,244,0


In [7]:
data["datetime"] = pd.to_datetime(data["datetime"])
data["datetime-1"] = pd.to_datetime(data["datetime-1"])

In [8]:
data.insert(loc=7, column="timestep", value=data["datetime"]-data["datetime-1"])
data.head()

,target,county,is_business,product_type,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
0,0.713,0,0,1,0,2021-09-01 00:00:00,NaT,NaT,0,0,0
1,96.590,0,0,1,1,2021-09-01 00:00:00,2021-09-01 00:00:00,0 days 00:00:00,0,1,0
122,1.132,0,0,1,0,2021-09-01 01:00:00,2021-09-01 00:00:00,0 days 01:00:00,0,122,0
123,77.691,0,0,1,1,2021-09-01 01:00:00,2021-09-01 01:00:00,0 days 00:00:00,0,123,0
244,0.490,0,0,1,0,2021-09-01 02:00:00,2021-09-01 01:00:00,0 days 01:00:00,0,244,0


In [9]:
# identify discontinuous series, i.e., those which contain a timestep different from 1 hour
max_timestep_per_unit = data[["prediction_unit_id", "timestep"]].groupby(by="prediction_unit_id").max()
discontinuous_series = max_timestep_per_unit[max_timestep_per_unit["timestep"]!=pd.Timedelta(1, "h")].index
max_timestep_per_unit.loc[discontinuous_series]

,timestep
prediction_unit_id,
21,31 days 01:00:00
26,62 days 01:00:00
41,173 days 01:00:00
44,31 days 01:00:00
47,63 days 01:00:00
68,76 days 01:00:00


In [ ]:
# drop discontinuous series
data = data[~data["prediction_unit_id"].isin(discontinuous_series)]

In [11]:
data["prediction_unit_id"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 42, 43, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

# Missing values

In [12]:
data.isna().sum()

target                496
county                  0
is_business             0
product_type            0
is_consumption          0
datetime                0
datetime-1              1
timestep                1
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64

In [13]:
data[data["target"].isna()].head()

,target,county,is_business,product_type,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
178938,NaN,0,0,1,0,2021-10-31 03:00:00,2021-10-31 02:00:00,0 days 01:00:00,60,178938,0
178939,NaN,0,0,1,1,2021-10-31 03:00:00,2021-10-31 03:00:00,0 days 00:00:00,60,178939,0
634866,NaN,0,0,1,0,2022-03-27 03:00:00,2022-03-27 02:00:00,0 days 01:00:00,207,634866,0
634867,NaN,0,0,1,1,2022-03-27 03:00:00,2022-03-27 03:00:00,0 days 00:00:00,207,634867,0
1332456,NaN,0,0,1,0,2022-10-30 03:00:00,2022-10-30 02:00:00,0 days 01:00:00,424,1332456,0


In [14]:
mask = ((data["datetime"] <= "2021-10-31 04:00:00")
        & (data["datetime"] >= "2021-10-31 02:00:00")
        & (data["prediction_unit_id"] == 0))
data[mask]

,target,county,is_business,product_type,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
178812,0.000,0,0,1,0,2021-10-31 02:00:00,2021-10-31 01:00:00,0 days 01:00:00,60,178812,0
178813,166.141,0,0,1,1,2021-10-31 02:00:00,2021-10-31 02:00:00,0 days 00:00:00,60,178813,0
178938,NaN,0,0,1,0,2021-10-31 03:00:00,2021-10-31 02:00:00,0 days 01:00:00,60,178938,0
178939,NaN,0,0,1,1,2021-10-31 03:00:00,2021-10-31 03:00:00,0 days 00:00:00,60,178939,0
179064,0.000,0,0,1,0,2021-10-31 04:00:00,2021-10-31 03:00:00,0 days 01:00:00,60,179064,0
179065,174.856,0,0,1,1,2021-10-31 04:00:00,2021-10-31 04:00:00,0 days 00:00:00,60,179065,0


We split consumption and production data to fill NaNs using the fill-forward method.

In [15]:
production = data.loc[data["is_consumption"] == 0, ["target", "datetime", "prediction_unit_id"]]
consumption = data.loc[data["is_consumption"] == 1, ["target", "datetime", "prediction_unit_id"]]

## Consumption

In [16]:
mask = ((consumption["datetime"] <= "2021-10-31 04:00:00")
        & (consumption["datetime"] >= "2021-10-31 02:00:00")
        & (consumption["prediction_unit_id"] == 0))
consumption[mask]

,target,datetime,prediction_unit_id
178813,166.141,2021-10-31 02:00:00,0
178939,NaN,2021-10-31 03:00:00,0
179065,174.856,2021-10-31 04:00:00,0


In [ ]:
nan_consumption_indices = consumption[consumption["target"].isna()].index
nan_consumption_indices

Index([ 178939,  634867, 1332457, 1806253,  178941,  634869, 1332459, 1806255,
        178943,  634871,
       ...
       1806379,  634891, 1332481, 1806277,  634909, 1332499, 1806295,  634965,
       1332557, 1806351],
      dtype='int64', length=248)

In [18]:
consumption.isna().sum()

target                248
datetime                0
prediction_unit_id      0
dtype: int64

In [19]:
consumption.ffill(inplace=True)
consumption.isna().sum()

target                0
datetime              0
prediction_unit_id    0
dtype: int64

In [20]:
consumption[mask]

,target,datetime,prediction_unit_id
178813,166.141,2021-10-31 02:00:00,0
178939,166.141,2021-10-31 03:00:00,0
179065,174.856,2021-10-31 04:00:00,0


## Production

In [21]:
mask = ((production["datetime"] <= "2021-10-31 04:00:00")
        & (production["datetime"] >= "2021-10-31 02:00:00")
        & (production["prediction_unit_id"] == 0))
production[mask]

,target,datetime,prediction_unit_id
178812,0.0,2021-10-31 02:00:00,0
178938,NaN,2021-10-31 03:00:00,0
179064,0.0,2021-10-31 04:00:00,0


In [ ]:
nan_production_indices = production[production["target"].isna()].index
nan_production_indices

Index([ 178938,  634866, 1332456, 1806252,  178940,  634868, 1332458, 1806254,
        178942,  634870,
       ...
       1806378,  634890, 1332480, 1806276,  634908, 1332498, 1806294,  634964,
       1332556, 1806350],
      dtype='int64', length=248)

In [23]:
production.isna().sum()

target                248
datetime                0
prediction_unit_id      0
dtype: int64

In [24]:
production.ffill(inplace=True)
production.isna().sum()

target                0
datetime              0
prediction_unit_id    0
dtype: int64

In [25]:
production[mask]

,target,datetime,prediction_unit_id
178812,0.0,2021-10-31 02:00:00,0
178938,0.0,2021-10-31 03:00:00,0
179064,0.0,2021-10-31 04:00:00,0


## Filling NaNs

In [ ]:
nan_production_indices

Index([ 178938,  634866, 1332456, 1806252,  178940,  634868, 1332458, 1806254,
        178942,  634870,
       ...
       1806378,  634890, 1332480, 1806276,  634908, 1332498, 1806294,  634964,
       1332556, 1806350],
      dtype='int64', length=248)

In [ ]:
nan_consumption_indices

Index([ 178939,  634867, 1332457, 1806253,  178941,  634869, 1332459, 1806255,
        178943,  634871,
       ...
       1806379,  634891, 1332481, 1806277,  634909, 1332499, 1806295,  634965,
       1332557, 1806351],
      dtype='int64', length=248)

In [ ]:
nan_indices = nan_production_indices.join(nan_consumption_indices, how="outer")

In [29]:
data.loc[nan_indices, "target"].isna().sum() == data["target"].isna().sum()

np.True_

In [ ]:
data.loc[nan_consumption_indices, "target"] = consumption.loc[nan_consumption_indices, "target"]
data.loc[nan_production_indices, "target"] = production.loc[nan_production_indices, "target"]
data["target"].isna().sum()

np.int64(0)

# Downcasting

In [31]:
data.dtypes

target                        float64
county                          int64
is_business                     int64
product_type                    int64
is_consumption                  int64
datetime               datetime64[ns]
datetime-1             datetime64[ns]
timestep              timedelta64[ns]
data_block_id                   int64
row_id                          int64
prediction_unit_id              int64
dtype: object

In [32]:
int_columns = list(data.dtypes[data.dtypes == np.int64].index)
float_columns = list(data.dtypes[data.dtypes == np.float64].index)

In [33]:
int_columns

['county',
 'is_business',
 'product_type',
 'is_consumption',
 'data_block_id',
 'row_id',
 'prediction_unit_id']

In [34]:
float_columns

['target']

In [35]:
for col in int_columns:
    data[col] = pd.to_numeric(data[col], downcast="unsigned")

In [36]:
for col in float_columns:
    data[col] = pd.to_numeric(data[col], downcast="float")

In [37]:
data.dtypes

target                        float32
county                          uint8
is_business                     uint8
product_type                    uint8
is_consumption                  uint8
datetime               datetime64[ns]
datetime-1             datetime64[ns]
timestep              timedelta64[ns]
data_block_id                  uint16
row_id                         uint32
prediction_unit_id              uint8
dtype: object

# Train-Test split

In [38]:
data = data.sort_values(by="datetime")
data

,target,county,is_business,product_type,is_consumption,datetime,datetime-1,timestep,data_block_id,row_id,prediction_unit_id
0,0.713000,0,0,1,0,2021-09-01 00:00:00,NaT,NaT,0,0,0
87,21.099001,11,0,1,1,2021-09-01 00:00:00,2021-09-01 00:00:00,0 days 00:00:00,0,87,43
86,0.000000,11,0,1,0,2021-09-01 00:00:00,2023-05-31 23:00:00,-638 days +01:00:00,0,86,43
85,927.348999,10,1,3,1,2021-09-01 00:00:00,2021-09-01 00:00:00,0 days 00:00:00,0,85,42
84,0.000000,10,1,3,0,2021-09-01 00:00:00,2023-05-15 23:00:00,-622 days +01:00:00,0,84,42
...,...,...,...,...,...,...,...,...,...,...,...
2018277,1307.526978,5,1,3,1,2023-05-31 23:00:00,2023-05-31 23:00:00,0 days 00:00:00,637,2018277,23
2018276,0.396000,5,1,3,0,2023-05-31 23:00:00,2023-05-31 22:00:00,0 days 01:00:00,637,2018276,23
2018305,58.411999,10,0,1,1,2023-05-31 23:00:00,2023-05-31 23:00:00,0 days 00:00:00,637,2018305,38
2018311,822.236023,10,1,3,1,2023-05-31 23:00:00,2023-05-31 23:00:00,0 days 00:00:00,637,2018311,42


In [ ]:
data.iloc[int(len(data) * 0.80)]

target                                0.0
county                                  9
is_business                             0
product_type                            3
is_consumption                          0
datetime              2023-01-26 05:00:00
datetime-1            2023-01-26 04:00:00
timestep                  0 days 01:00:00
data_block_id                         512
row_id                            1618384
prediction_unit_id                     35
Name: 1618384, dtype: object

In [40]:
train = data[data["datetime"] <= "2023-01-26 05:00:00"]
test = data[data["datetime"] > "2023-01-26 05:00:00"]
print(train.shape)
print(test.shape)

(1521300, 11)
(380268, 11)


In [41]:
len(data) == len(train) + len(test)

True

In [42]:
len(train) / len(data)

0.8000239802100162

In [43]:
len(test) / len(data)

0.19997601978998383

In [44]:
# production_train = train[train["is_consumption"] == 0]
# production_test = test[test["is_consumption"] == 0]

In [45]:
# consumption_train = train[train["is_consumption"] == 1]
# consumption_test = test[test["is_consumption"] == 1]

# Save cleaned data

In [46]:
import os

In [47]:
if "preprocessed" not in os.listdir("../data"):
    os.mkdir("../data/preprocessed")

In [48]:
# TODO: save in an other format to preserve data types
train.to_csv("../data/preprocessed/train.csv", index=False)
# production_train.to_csv("../data/preprocessed/production_train.csv", index=False)
# consumption_train.to_csv("../data/preprocessed/consumption_train.csv", index=False)

In [49]:
test.to_csv("../data/preprocessed/test.csv", index=False)
# production_test.to_csv("../data/preprocessed/production_test.csv", index=False)
# consumption_test.to_csv("../data/preprocessed/consumption_test.csv", index=False)

In [50]:
os.listdir("../data/preprocessed")

['consumption_train.csv',
 'test.csv',
 'consumption_test.csv',
 'production_train.csv',
 'train.csv',
 'production_test.csv']

In [51]:
# Load data to AWS S3 (Optional)
if config["s3_bucket"]:
    s3_client = boto3.client('s3')
    with open("../data/preprocessed/train.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/preprocessed/train.csv")
    with open("../data/preprocessed/test.csv", "rb") as file:
        s3_client.upload_fileobj(file, config["s3_bucket"], "data/preprocessed/test.csv")

In [52]:
# Check that data has been uploaded correctly
if config["s3_bucket"]:
    response = s3_client.list_objects(
        Bucket=config["s3_bucket"])
    for obj in response.get("Contents"):
        print(obj.get("Key"))